In [ ]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
import patchify
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image as skimg
import imgaug as ia
import imgaug.augmenters as iaa
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_V2_Weights
import torch.optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.insert(1, '/home/prakharug/AFO')
sys.path.insert(1, '/home/prakharug/AFO/pycoco')
from pycoco.engine import train_one_epoch, evaluate

In [ ]:
instance_toimage = []

for im_name in os.listdir("./dataset1/images"):
    tmplist = []
    lent = len(im_name[:-4])
    print(im_name,im_name[:lent])
    imgtmp = cv2.imread("./dataset1/images/"+im_name,1).transpose(2,0,1)
    #print(imgtmp[0].shape)
    tmplist.append(imgtmp[0])
    tmplist.append(imgtmp[1])
    tmplist.append(imgtmp[2])
    for gt_name in os.listdir("./dataset1/ground_truths1"):
        #print(gt_name[10+lent],gt_name[10:10+lent])
        if gt_name[10:10+lent]==im_name[:lent] and gt_name[10+lent]=="_":
            mask = np.array(cv2.imread("./dataset1/ground_truths1/"+gt_name,0))
            # mask = (mask > 0).astype(np.uint8) 
            tmplist.append(mask)
    tmplist = np.array(tmplist)
    print(tmplist.shape)
    instance_toimage.append(tmplist)

for im_name in os.listdir("./dataset1/test"):
    tmplist = []
    lent = len(im_name[:-4])
    print(im_name,im_name[:lent])
    imgtmp = cv2.imread("./dataset1/test/"+im_name,1).transpose(2,0,1)
    #print(imgtmp[0].shape)
    tmplist.append(imgtmp[0])
    tmplist.append(imgtmp[1])
    tmplist.append(imgtmp[2])
    for gt_name in os.listdir("./dataset1/ground_truths1"):
        #print(gt_name[10+lent],gt_name[10:10+lent])
        if gt_name[10:10+lent]==im_name[:lent] and gt_name[10+lent]=="_":
            mask = np.array(cv2.imread("./dataset1/ground_truths1/"+gt_name,0))
            # mask = (mask > 0).astype(np.uint8) 
            tmplist.append(mask)
    tmplist = np.array(tmplist)
    print(tmplist.shape)
    instance_toimage.append(tmplist)

In [ ]:
cell_img = []
j = 1
for mksl in instance_toimage:
    #print(len(mksl))
    for mksin in range(3,len(mksl)):
        x,y,w,h = cv2.boundingRect(mksl[mksin])
        if x*y*w*h==0 or abs(x+w-mksl.shape[2])<=50 or abs(y+h-mksl.shape[1])<=50:
            print("bruh")
            continue
        else:
            stck1 = mksl[0]*(mksl[mksin]/255)
            stck2 = mksl[1]*(mksl[mksin]/255)
            stck3 = mksl[2]*(mksl[mksin]/255)
            print(stck1.shape)
            new_img = np.array([stck1,stck2,stck3],np.uint8)[:,y:y+h,x:x+w]
            print(new_img.shape)
            cell_img.append(new_img)
            #cv2.imwrite('./cell_iso1/jsr'+str(j)+'.png',new_img.transpose(1,2,0))
            j+=1
            #break
            #cv2.waitkey(0)
            #cv2.destroyAllindows

In [11]:
def blend_non_transparent(face_img, overlay_img):
    # Let's find a mask covering all the non-black (foreground) pixels
    # NB: We need to do this on grayscale version of the image
    #print("overlay img ",overlay_img.shape)
    gray_overlay = cv2.cvtColor(overlay_img, cv2.COLOR_BGR2GRAY)
    overlay_mask = cv2.threshold(gray_overlay, 1, 255, cv2.THRESH_BINARY)[1]

    # Let's shrink and blur it a little to make the transitions smoother...
    #overlay_mask = cv2.erode(overlay_mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))
    #overlay_mask = cv2.blur(overlay_mask, (3, 3))

    # And the inverse mask, that covers all the black (background) pixels
    background_mask = (255) - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [12]:
lob = 1000
for iml in instance_toimage:
    # cv2.imwrite("./dataset2/images/"+str(lob)+'.png',iml[0:3].transpose(1,2,0))
    # for k in range(0,iml.shape[0]-3):
    #     cv2.imwrite("./dataset2/ground_truths/"+str(lob)+'_'+str(k+1)+'.png',np.array([iml[k+3]]).transpose(1,2,0))
    # lob+=1
    i=0
    if(iml.shape[0]<=5):
        continue
    while i<=1:
        i+=1
        tmpl = iml.copy()
        cnt=0
        for j in range(0,10):
            paster = False
            idx = random.randint(0,len(cell_img)-1)
            rtrnd=random.randint(0,4)
            seq = iaa.Sequential([
            iaa.WithHueAndSaturation([
            iaa.WithChannels(0, iaa.Add((-30, 10))),
            iaa.WithChannels(1, [
                iaa.Multiply((0.5, 1.5)),
                iaa.LinearContrast((0.75, 1.25))
                ])
            ])
            ,
            iaa.Rot90((rtrnd), keep_size=False)
            ])

            # print(cell_img[idx].shape)
            test = cell_img[idx].transpose(1,2,0)
            cell_paste = seq(images=[test])
            cell_paste = cell_paste[0].transpose(2,0,1)
            #cell_paste = cell_img[idx]
            k_w = cell_paste.shape[2]
            k_h = cell_paste.shape[1]
            tryer=0
            o_w=0
            o_h=0
            while paster==False and tryer<=5:
                tryer+=1
                o_w = random.randint(0,tmpl.shape[2]-k_w)
                o_h = random.randint(0,tmpl.shape[1]-k_h)
                patch_img = tmpl[0:3,o_h:o_h+k_h,o_w:o_w+k_w]
                instances = tmpl[3:,o_h:o_h+k_h,o_w:o_w+k_w]
                instances = instances.transpose(1,2,0)
                for a in range(instances.shape[2]):
                    dispim = instances[:,:,a]
                    #cv2.imwrite('writer.jpg',dispim)
                    if np.all(dispim == 0):
                        paster = True
                        continue
                    else:
                        paster=False
                        break
                    
            if paster==True:
                back = np.zeros([3,tmpl.shape[1],tmpl.shape[2]],np.uint8)
                back[:,o_h:o_h+k_h,o_w:o_w+k_w] = cell_paste
                grao = cv2.cvtColor(back.transpose(1,2,0), cv2.COLOR_BGR2GRAY)
                grao = (grao>0)*255
                grao = np.array([grao])
                #print(cell_paste.shape,k_h)
                #cv2.imwrite('jsr.png',tmpl[0:3].transpose(1,2,0))
                #print(type(tmpl[0:3]),type(back))
                tmpl[0:3] = blend_non_transparent(tmpl[0:3].transpose(1,2,0),back.transpose(1,2,0)).transpose(2,0,1)
                #cv2.imwrite('jsr.png',tmpl[0:3].transpose(1,2,0))
                #cv2.imwrite('comp.png',cell_paste.transpose(1,2,0))
                #print(tmpl.shape,grao.shape)
                tmpl = np.append(tmpl,grao,0)
                #print(tmpl.shape,grao.shape)
                cnt+=1
        if cnt>=4:
            print('hello')
            cv2.imwrite("./dataset2/images/"+str(lob)+'.png',tmpl[0:3].transpose(1,2,0))
            for k in range(0,tmpl.shape[0]-3):
                cv2.imwrite("./dataset2/ground_truths/"+str(lob)+'_'+str(k+1)+'.png',np.array([tmpl[k+3]]).transpose(1,2,0))
            lob+=1
        else:
            i-=1

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hell

In [13]:
for im_name in os.listdir("./submission/images"):
    tmplist = []
    lent = len(im_name[:-4])
    print(im_name,im_name[:lent])
    imgtmp = cv2.imread("./submission/images/"+im_name,1)
    imgtmp = (imgtmp > 0)*255
    cv2.imwrite("./submission/x/"+im_name[0:-4]+".png",imgtmp)

2006.bmp 2006
1806.bmp 1806
1893.bmp 1893
9711.bmp 9711
9764.bmp 9764
9618.bmp 9618
608.bmp 608
9392.bmp 9392
2294.bmp 2294
9898.bmp 9898
9515.bmp 9515
514.bmp 514
2133.bmp 2133
9753.bmp 9753
1988.bmp 1988
9655.bmp 9655
9716.bmp 9716
205.bmp 205
2103.bmp 2103
105.bmp 105
9787.bmp 9787
1945.bmp 1945
1711.bmp 1711
9524.bmp 9524
1707.bmp 1707
2411.bmp 2411
9578.bmp 9578
1761.bmp 1761
1995.bmp 1995
9495.bmp 9495
9332.bmp 9332
2407.bmp 2407
9586.bmp 9586
1840.bmp 1840
9769.bmp 9769
1828.bmp 1828
9404.bmp 9404
9813.bmp 9813
9894.bmp 9894
9663.bmp 9663
1969.bmp 1969
2124.bmp 2124
9437.bmp 9437
1754.bmp 1754
301.bmp 301
1903.bmp 1903
605.bmp 605
9330.bmp 9330
2256.bmp 2256
9763.bmp 9763
2134.bmp 2134
2084.bmp 2084
9355.bmp 9355
9675.bmp 9675
2376.bmp 2376
2316.bmp 2316
9843.bmp 9843
2409.bmp 2409
9808.bmp 9808
9807.bmp 9807
2232.bmp 2232
1927.bmp 1927
1827.bmp 1827
9288.bmp 9288
2173.bmp 2173
2119.bmp 2119
2235.bmp 2235
2219.bmp 2219
2113.bmp 2113
9414.bmp 9414
2247.bmp 2247
1892.bmp 1892
2120

In [ ]:
for im_name in os.listdir("./test/images2"):
    tmplist = []
    lent = len(im_name[:-4])
    print(im_name,im_name[:lent])
    imgtmp = cv2.imread("./test/images2/"+im_name,1)
    #imgtmp = (imgtmp > 0)*255
    cv2.imwrite("./test/images/"+im_name[0:-4]+".png",imgtmp)

In [ ]:
for im_name in os.listdir("./dataset1/ground_truths1"):
    tmplist = []
    lent = len(im_name[10:-4])
    print(im_name,im_name[10:-4])
    imgtmp = cv2.imread("./dataset1/ground_truths1/"+im_name,1)
    #imgtmp = (imgtmp > 0)*255
    cv2.imwrite("./dataset1/ground_truths/"+im_name[10:-4]+".png",imgtmp)